Introduction

In [ ]:
# EDA
    #Finished Just Comment code, fix typos, reformat analysis 
# Cluster
    # PCA / Kmeans # Do own implementation too?

    
#### Fix up Other Sections?   
    # Fix dist plots
    # Fix prob plots
    # Fix box plot labels?
    # Add in background informations for these variables
    # When doing analyses compare the average results
    # Include mean / median
    # Include more analysis on boxplots
    # Remove team kills 

    
# Possible alterations 3D, 2D, PCA with all features, clean up training and testing set, look into validation sets to see if any bias!!!!!!!!!
# CLustering is finished
    
# Write up:
# Introduction
# EDA
# Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import numpy as np
import pandas as pd
import seaborn as sns
import xlrd
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
pd.options.mode.chained_assignment = None

# Set an offline environment for plotly
init_notebook_mode(connected=True)

Data Preprocessing / Feature Engineering

Let us begin by reading in the CSV file containing the data, and examining the data contents such as the number of features and rows. It seems there are 152 column entries (features) and 87898 row entries (number of samples).

In [ ]:
# Read in CSV
orig = pd.read_csv(r"C:\Users\sulma\Downloads\Kaggle Datasets\PUBG\PUBG-hack-detection-master\PUBG_Player_Statistics.csv")
# Examine data contents
orig.info()
orig.head()

Now, let us remove information that does not pertain to our goal of clustering solo player behavior. 

Remove player_name, tracker_id, duo, and squad columns. This can be achieved by removing all columns after the 52nd. Also, create a new feature that combines the walking and riding distance.

In [ ]:
## df Preprocessing
# Create a copy of the dataframe
df = orig.copy()
cols = np.arange(52, 152, 1)
# Drop columns after the 52nd index
df.drop(df.columns[cols],axis=1,inplace=True)
# Drop player_name and tracker id
df.drop(df.columns[[0, 1]],axis=1,inplace=True)
# Drop Knockout and Revives
df.drop(df.columns[[49]],axis=1,inplace=True)
df.drop(columns = ['solo_Revives'], inplace=True)
# Let's drop the string solo from all strings
df.rename(columns= lambda x: x.lstrip('solo_').rstrip(''), inplace=True)
# Combine a few columns 
df['TotalDistance'] = df['WalkDistance'] + df['RideDistance']
df['AvgTotalDistance'] = df['AvgWalkDistance'] + df['AvgRideDistance']



We will split the data into three sets: train, dev, and test set.

In [ ]:
# Create train and test set
train, test = train_test_split(df, test_size=0.1)
dev, test = train_test_split(test, test_size = 0.5)
data = train

print(len(train))
print(len(dev))
print(len(test))

It is important we go through the final output to make sure that are data preprocessing is complete. And it looks great!

In [ ]:
with pd.option_context('display.max_columns', 52):
    print(data.describe(include='all'))



In the next section, we'll go over various representations of the features. Starting off with univarate plots and then observing bivariate relationships.

Univariate Plots 
Let's examine a select few features

Continous Representations

We will select the following features because they are the more prominent predictors from domain experience

From what we observe, the distribution plots imply that all of these features are right-skewed except for the  AvgSurvivalTime, which appears to be following a normal distribution.

In [ ]:
## Histogram Plots
# Creating axes for plots
f, axes = plt.subplots(3, 4, figsize=(30,20), sharex=False)
# Distplots
sns.distplot(data["Kills"],  color="skyblue", ax=axes[0, 0])
sns.distplot(data["KillDeathRatio"] , color="olive", ax=axes[0, 1])
sns.distplot(data["Wins"] , color="gold", ax=axes[1, 0])
sns.distplot(data["WinRatio"] , color="teal", ax=axes[1, 1])
# sns.distplot(data["Top10Ratio"] , color="orange", ax=axes[2, 1])
sns.distplot(data["HeadshotKills"] , color="blue", ax=axes[0, 2])
sns.distplot(data["HeadshotKillRatio"] , color="red", ax=axes[1, 2])
sns.distplot(data["DamageDealt"] , color="purple", ax=axes[0, 3])
sns.distplot(data["DamagePg"] , color="magenta", ax=axes[1, 3])
sns.distplot(data["RoundsPlayed"] , color="silver", ax=axes[2, 0])
#sns.distplot(data["TimeSurvived"] , color="orange", ax=axes[2, 1])
sns.distplot(data["AvgSurvivalTime"] , color="orange", ax=axes[2, 1])
sns.distplot(data['TotalDistance'] , color="green", ax=axes[2, 2])
sns.distplot(data["AvgTotalDistance"] , color="darkblue", ax=axes[2, 3])


We can support the claim of it being normal by also examining the probability plot and see if it follows the lines

In [ ]:
## Probability Plots
# Combat
ax1 = plt.subplot(221)
stats.probplot( data["Kills"],plot=sns.mpl.pyplot) 
ax2 = plt.subplot(222)
stats.probplot( data["KillDeathRatio"],plot=sns.mpl.pyplot) 
ax3 = plt.subplot(223)
stats.probplot( data["Wins"],plot=sns.mpl.pyplot) 
ax4 = plt.subplot(224)
stats.probplot( data["WinRatio"],plot=sns.mpl.pyplot)
fig = plt.figure()
ax_1 = fig.add_subplot(221)
stats.probplot( data["HeadshotKills"],plot=sns.mpl.pyplot) 
ax_2 = fig.add_subplot(222)
stats.probplot( data["HeadshotKillRatio"] ,plot=sns.mpl.pyplot)
ax_3 = fig.add_subplot(223)
stats.probplot( data["DamageDealt"],plot=sns.mpl.pyplot) 
ax_4 = fig.add_subplot(224)
stats.probplot( data["DamagePg"],plot=sns.mpl.pyplot)
fig = plt.figure()
ax_31 = fig.add_subplot(221)
stats.probplot( data["RoundsPlayed"] ,plot=sns.mpl.pyplot)
ax_32 = fig.add_subplot(222)
stats.probplot( data["AvgSurvivalTime"] ,plot=sns.mpl.pyplot)
ax_33 = fig.add_subplot(223)
stats.probplot( data['TotalDistance'],plot=sns.mpl.pyplot) 
ax_34 = fig.add_subplot(224)
stats.probplot( data["AvgTotalDistance"],plot=sns.mpl.pyplot)


Now, that we have gone through the basic understanding of the continous representation of the data. Let's discretize the data into specific intervals and see how the population is spread. What we imply with discretizing is we want to convert a feature from numerical into categorical, this is very important for conducting boxplots or violin plots, as we don't want to look at all of the points but a select few intervals!

Discretized

Our function below will take a feature, discretize it, and then plots the representation of it.

In [ ]:
###Creates a seaborn count plot by discretizing a column to change into an interval (category)
## Use 
# 1) Discretize a feature: convert it from numerical to categorical
# 2) Plot a countplot using seaborn
# 3) Annotate the frequency of each interval
## Function parameters:
# data == dataframe
# discretized == new feature (category)
# cut == feature that is being
# converted into a category
# plot_label = label of the x-axis and title
# rotation = rotation of the x-tick marks
# bins == the intervals
# labels == string representations of the bins

def CountPlot(data, discretized, cut, title_label, plot_xlabel, rotation, bins, labels):
    # Discretize the data
    data[discretized] = pd.cut(data[cut], bins = bins, labels = labels)
    
    # Plot
    dfWIM = data
    plt.style.use('ggplot')
    plt.figure(figsize=(15,10))
    ax = sns.countplot(x=discretized, data=dfWIM, order= labels)
    plt.title('Distribution of'+ ' ' + title_label)
    plt.xlabel(plot_xlabel)
    plt.xticks(rotation = rotation, fontsize=10)

    # Annotate the frequencies of each interval
    ncount = len(dfWIM)
    for p in ax.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha='center', va='bottom', rotation = 90) # set the alignment of the text
    return plt.show()


Kills / Kill Death Ratio

Distribution of Kills:
* Most players are in the range of 0 - 9 kills, which is about 9.1% of the data.

Distribution of Kill Death Ratio
* Most players are in intervals of 0.60 - 0.79, 0.80 - 0.99, and 1.00 - 1.19. For reference, a KDR of 1.0 equates to that for every death you accomplished one kill.

In [ ]:
# Distribution of Kills
bins = [i for i in range(0, 610, 10)] + [5033]
labels = [str(i)+ '-' + str(i+9) for i in range(0,600,10)] + ['600.0-5033']
CountPlot(data = data, discretized = 'Kills_Category', cut = 'Kills'
          , title_label = 'Kills' , plot_xlabel = 'Kills'
          , rotation = 90 , bins = bins, labels = labels)

# Distribution of KDR(Kill Death Ratio)
bins =  [i for i in np.arange(0, 9.2, 0.2)] + [100]
labels = ["{0:.2f}".format(i)+ '-' + "{0:.2f}".format(i+0.19) for i in np.arange(0, 9.0, 0.2)] + ['9.0-100']
CountPlot(data = data, discretized = 'KillDeathRatio_Category', cut = 'KillDeathRatio'
          , title_label = 'Kill-Death Ratio', rotation = 45 ,plot_xlabel = 'Kill-Death Ratio'
          , bins = bins, labels = labels)


Distribution of Headshots:
* Most players are in the range of 0 - 9 headshots, which is about 34.6% of the data.

Distribution of Headshot Ratio:
* Most players are in intervals of 0.150 - 0.199 and 0.200 - 0.249 For reference, a HKR of 1.0 equates to that for every kill you accomplished one headshot.

In [ ]:
## Headshots and Headshot Ratio
# Headshots
bins = [i for i in range(0, 150, 10)] + [1500]
labels = [str(i)+ '-' + str(i+9) for i in range(0,140,10)] + ['140-1500']
CountPlot(data = data, discretized = 'HeadshotKills_Category'
          , cut = 'HeadshotKills', title_label = 'Headshots' , plot_xlabel = 'Headshots'
          , rotation = 0 , bins = bins, labels = labels)

# Headshot Kill Ratio
bins =  [i for i in np.arange(0, 0.55, 0.050)] + [1.1]
labels = ["{0:.3f}".format(i)+ '-' + "{0:.3f}".format(i+0.049) for i in np.arange(0, 0.50, 0.050)] + ['0.50-1.00']
CountPlot(data = data, discretized = 'HeadshotKillRatio_Category'
         , cut = 'HeadshotKillRatio', title_label = 'Headshot-Kill Ratio'
         , plot_xlabel = 'Headshot-Kill Ratio' , rotation = 0 , bins = bins, labels = labels)


Wins And Top 10

Distribution of Wins:
* Most players are in the range of 0 - 9 wins, which is about 25.9% of the data.

Distribution of Win Ratio:
* Most players are in the interval of 1.00  - 1.99 (%). A 1.0% win ratio is analagous to every 100 games, one win is achieved.

In [ ]:
## Wins and Win Ratio
# Wins
bins =  [i for i in range(0, 16, 1)] + [106]
labels = [str(i) for i in range(0,15,1)] + ['15-106']
CountPlot(data = data, discretized = 'Wins_Category'
         , cut = 'Wins', title_label = 'Wins' , plot_xlabel = 'Wins' 
         , rotation = 0 , bins = bins, labels = labels)

# Win Ratio
bins = [i for i in np.arange(0, 51, 1)] + [100]
labels = ["{0:.2f}".format(i)+ '-' + "{0:.2f}".format(i+0.99) for i in np.arange(0, 50, 1)] + ['50.00-100']
CountPlot(data = data, discretized = 'WinsRatio_Category'
         , cut = 'WinRatio', title_label = 'Win Ratio', plot_xlabel = 'Win Ratio (%)' 
         , rotation = 90 , bins = bins, labels = labels)

Distribution of Top 10:
* Most players have not achieved a top 10 finish, which is roughly 6.9% of the data.

Distribution of Top 10 Ratio:
* Most players are in intervals of 9.00 - 9.99. Implies that you make 9 top 10 finishes out of 100 rounds played.

In [ ]:
## Top 10s and Top10 Ratio
# Top 10s
bins = [i for i in np.arange(0, 70, 1)] + [386]
labels = [ str(i) for i in np.arange(0, 69, 1)] + ['69-386']
CountPlot(data = data, discretized = 'Top10s_Category'
         , cut = 'Top10s', title_label = 'Top 10s',  plot_xlabel = 'Top 10s' 
         , rotation = 90 , bins = bins, labels = labels)

# Top10 Ratio
bins =  [i for i in np.arange(0, 70, 1)] + [100]
labels = ["{0:.2f}".format(i)+ '-' + "{0:.2f}".format(i+0.99) for i in np.arange(0, 69, 1)] + ['69.00-100']
CountPlot(data = data, discretized = 'Top10Ratio_Category'
         , cut = 'Top10Ratio', title_label = 'Top 10 Ratios' , plot_xlabel = 'Top 10 Ratio (%)' 
         , rotation = 90 , bins = bins, labels = labels)


Distance

Distribution of Total Distance:
* Most players are in the range of 0 - 19999 miles, which is about 12.1% of the data.
* The average man will travel 110,000 miles in his lifetime, that is roughly 6x the reported amount from the majority of players.

Distribution of Average Distance per game:
* Most data is represented in the middle portion of 1800 - 3000 miles
* The average man will travel 1,000 miles (driving) + 3.7 miles (walking). The order of magnitudes are the same 

In [ ]:
## Total Distance and Average Distance per game
# Total Distance
bins =  [i for i in range(0, 1260000 , 20000)] + [6.490221e+06]
labels = [str(i)+ '-' + str(i+19999) for i in range(0,1240000,20000)] + ['1240000-6.490221e+06']
CountPlot(data = data, discretized = 'TotalDistance_Category'
         , cut = 'TotalDistance', title_label = 'Total Distance', plot_xlabel = 'Total Distance (m)' 
         , rotation = 90 , bins = bins, labels = labels)

# Average Distance Per Game
bins = [i for i in np.arange(0, 7100 , 100)] + [77000]
labels = [str(i)+ '-' + str(i+99) for i in np.arange(0,7000,100)] + ['7000-77000']
CountPlot(data = data, discretized = 'AvgTotalDistance_Category'
         , cut = 'AvgTotalDistance', title_label = 'Average Distance', plot_xlabel = 'Average Distance (m)' 
         , rotation = 90 , bins = bins, labels = labels)


Time Surived and Average Time Survived

Distribution of Total Time Survived:
* Most players are in the range of 0 - 9999 seconds, which is about 15.7% of the data.
* The average man will live 22,075,000 seconds in his lifetime.

Distribution of Average Time Survived Per Round:
* Most data is represented in the middle portion of 900 - 999 seconds.

In [ ]:
## Time Survived and Average Time Survived
# Time Survived
bins = [i for i in np.arange(0, 400000 , 10000)] + [1530000]
labels = [str(i) + '-' + str(i+9999) for i in np.arange(0, 390000, 10000)] + ['390000-1530000']
CountPlot(data = data, discretized = 'TimeSurvived_Category'
         , cut = 'TimeSurvived', title_label = 'Time Survived', plot_xlabel = 'Time Survived (s)' 
         , rotation = 90 , bins = bins, labels = labels)

# Average Time Survived Per Round
bins = [i for i in np.arange(0, 1800, 100)] + [2200]
labels = [str(i) + '-' + str(i+99) for i in np.arange(0, 1700 , 100)] + ['1610-2200']
CountPlot(data = data, discretized = 'AvgSurvivalTime_Category'
         , cut = 'AvgSurvivalTime', title_label = 'Average Time Survived', plot_xlabel = 'Average Time Survived (s)' 
         , rotation = 45 , bins = bins, labels = labels)


### Rounds Played and Damage Per Game

Distribution of Rounds Played:
* Most players are in the range of 0 - 9 rounds, which is about 17.1% of the data.


Distribution of Damage Per Round:
* Most data is represented in the middle portion of 130 - 139 (DMG per Round), which is 6.0%.

In [ ]:
## Rounds Played and Damage Per Round
# RoundsPlayed
bins = [i for i in range(0, 480 , 10)] + [1682]
labels = [str(i)+ '-' + str(i+9) for i in range(0,470,10)] + ['470 - 1682']

CountPlot(data = data, discretized = 'RoundsPlayed_Category'
         , cut = 'RoundsPlayed', title_label = 'Rounds Played', plot_xlabel = 'Rounds' 
         , rotation = 45 , bins = bins, labels = labels)

# Damage Dealt
bins = [i for i in range(0, 630 , 10)] + [2030]
labels = [str(i)+ '-' + str(i+9) for i in range(0,620,10)] + ['620 - 1230']
CountPlot(data = data, discretized = 'DamagePg_Category'
         , cut = 'DamagePg', title_label = 'Damage Per Game', plot_xlabel = 'Damage' 
         , rotation = 90 , bins = bins, labels = labels)


Bivariate Relations

To examine bivariate relationships, we will use scatter plots and box plots. In order for the number of box plots to be minimal, we will be discretizing the features into intervals. The function below takes an input feature and outputs a 2D scatter plot and a boxplot containing discretized intervals. But what are we comparing? In this case, let's examine the features in the previous section with win ratios to see if we can obtain any features for prediciting win ratios

It's also probably best to remove as much variability as possible by filtering out players that have lower than average number of rounds played

In [ ]:
# Remove samples with a lower than the mean amount of rounds played
train = train.drop(train[train['RoundsPlayed'] < train['RoundsPlayed'].mean()].index)
dev = dev.drop(dev[dev['RoundsPlayed'] < dev['RoundsPlayed'].mean()].index)
test = test.drop(test[test['RoundsPlayed'] < test['RoundsPlayed'].mean()].index)

In [ ]:
data = train

In [ ]:
data.corr(method = 'pearson')['WinRatio']

In [ ]:
###Creates a seaborn scatter and boxplot by discretizing a column to change into an interval (category)
## Use 
# 1) Discretize a feature: convert it from numerical to categorical
# 2) Plot a scatter plot
# 3) Plot a box plot
## Function parameters:
# data == dataframe
# discretized == new feature (category)
# cut == feature that is being
# bins == the intervals
# labels == string representations of the bins
# x = x_axis of scatterplot
# y = y_axis of scatterplot

def ScatterBoxPlot(data, discretized, cut, bins, labels, x, y):
    # Discretize the data
    data[discretized] = pd.cut(data[cut], bins = bins, labels = labels)
    
    # Give a numerical label to the category
    c = data[discretized].astype('category')
    d = dict(enumerate(c.cat.categories))
    box_code = discretized + '_' + 'code'
    level_back = discretized + '_' + 'level_back'
    data[box_code] = data[discretized].astype('category').cat.codes
    data[level_back] = data[box_code].map(d) # we need this to color our boxplot
    
    # Scatter
    fig = px.scatter(data, x=x , y=y , color = box_code)
    fig.show()

    # Boxplot
    fig = px.box(data, x=discretized , y=y , color= box_code, category_orders= {discretized: labels})
    fig.show()

There is no clear indication of a relationship as there is a ton of variability

In [ ]:
# Kills
bins = [i for i in range(0, 700, 100)] + [5600]
labels = [str(i)+ '-' + str(i+99) for i in range(0,600,100)] + ['600.0-5600']
ScatterBoxPlot(data = data, discretized = 'Kills_Box' , cut = 'Kills'
               , bins = bins , labels = labels, x = 'Kills', y = 'WinRatio')


In comparison with the kills, the KDR has variability, but there is a trend we can observe with win ratios. Higher KDAs tend to have higher win rates.

In [ ]:
# KDR
bins =  [i for i in np.arange(0, 10, 1)] + [110]
labels = ["{0:.2f}".format(i)+ '-' + "{0:.2f}".format(i+0.99) for i in np.arange(0, 9, 1)] + ['9.0-100']

ScatterBoxPlot(data = data, discretized = 'KillDeathRatio_Box' , cut = 'KillDeathRatio'
               , bins = bins , labels = labels, x = 'KillDeathRatio', y = 'WinRatio')

There is no clear indication of a relationship as there is a ton of variability

In [ ]:
# Headshot
bins = [i for i in range(0, 150, 10)] + [1500]
labels = [str(i)+ '-' + str(i+9) for i in range(0,140,10)] + ['140-1500']

ScatterBoxPlot(data = data, discretized = 'HeadshotKills_Box' , cut = 'HeadshotKills'
               , bins = bins , labels = labels, x = 'HeadshotKills', y = 'WinRatio')

There is no clear indication of a relationship as there is a ton of variability

In [ ]:
# Headshot Kill Ratio
bins =  [i for i in np.arange(0, 0.55, 0.050)] + [1.1]
labels = ["{0:.3f}".format(i)+ '-' + "{0:.3f}".format(i+0.049) for i in np.arange(0, 0.50, 0.050)] + ['0.50-1.00']
ScatterBoxPlot(data = data, discretized = 'HeadshotKillRatio_Box' , cut = 'HeadshotKillRatio'
               , bins = bins , labels = labels, x = 'HeadshotKillRatio', y = 'WinRatio')

There seems to be a clear trend of greater top 10 ratios lead to high win ratios.

In [ ]:
# Top 10s
bins =  [i for i in np.arange(0, 110, 10)]
labels = ["{0:.2f}".format(i)+ '-' + "{0:.2f}".format(i+9.99) for i in np.arange(0, 100, 10)] 
ScatterBoxPlot(data = data, discretized = 'Top10Ratio_Box' , cut = 'Top10Ratio'
               , bins = bins , labels = labels, x = 'Top10Ratio', y = 'WinRatio')

Higher Damage Per game leads to a higher win rate

In [ ]:
# Damage per game
bins = [i for i in range(0, 700 , 100)] + [2030]
labels = [str(i)+ '-' + str(i+9) for i in range(0,600,100)] + ['510 - 2030']
ScatterBoxPlot(data = data, discretized = 'DamagePg_Box' , cut = 'DamagePg'
               , bins = bins , labels = labels, x = 'DamagePg', y = 'WinRatio')


Rounds Played does not correlate with win ratio

In [ ]:
# Rounds Played
bins = [i for i in range(0, 550 , 50)] + [1682]
labels = [str(i)+ '-' + str(i+49) for i in range(0,500,50)] + ['470 - 1682']

ScatterBoxPlot(data = data, discretized = 'RoundsPlayed_Box' , cut = 'RoundsPlayed'
               , bins = bins , labels = labels, x = 'RoundsPlayed', y = 'WinRatio')

No clear indiciation of a trend

In [ ]:
# Time Survived
bins = [i for i in np.arange(0, 500000 , 100000)] + [1530000]
labels = [str(i) + '-' + str(i+99999) for i in np.arange(0, 400000, 100000)] + ['390000-1530000']
ScatterBoxPlot(data = data, discretized = 'TimeSurvived_Box' , cut = 'TimeSurvived'
               , bins = bins , labels = labels, x = 'TimeSurvived', y = 'WinRatio')

There is a linear relationship

In [ ]:
# Average Time Survived
bins = [i for i in np.arange(0, 2000, 200)] + [2200]
labels = [str(i) + '-' + str(i+199) for i in np.arange(0, 1800 , 200)] + ['1800-2200']
ScatterBoxPlot(data = data, discretized = 'AvgSurvivalTime_Box' , cut = 'AvgSurvivalTime'
               , bins = bins , labels = labels, x = 'AvgSurvivalTime', y = 'WinRatio')


No indication of a trend

In [ ]:
# Total Distance Survived
bins =  [i for i in range(0, 1400000 , 200000)] + [6.490221e+06]
labels = [str(i)+ '-' + str(i+199999) for i in range(0,1200000,200000)] + ['1200000-6500000']
ScatterBoxPlot(data = data, discretized = 'TotalDistance_Box' , cut = 'TotalDistance'
               , bins = bins , labels = labels, x = 'TotalDistance', y = 'WinRatio')

Too much variability to decide

In [ ]:
# Average Distance
bins = [i for i in np.arange(0, 8000 , 1000)] + [77000]
labels = [str(i)+ '-' + str(i+999) for i in np.arange(0,7000,1000)] + ['7000-77000']

ScatterBoxPlot(data = data, discretized = 'AvgTotalDistance_Box' , cut = 'AvgTotalDistance'
               , bins = bins , labels = labels, x = 'AvgTotalDistance', y = 'WinRatio')

Analysis

How many people who don't damage or kill anyone win games?

In [ ]:
Zero_Kills                = (data['Kills'] == 0).value_counts()
Zero_Damage               = (data['DamagePg'] == 0).value_counts() 
Zero_Kills_or_Damage     = ((data['Kills'] == 0) | (data['DamagePg'] == 0)).value_counts()
Zero_Kills_or_Damage_Win = (((data['Kills'] == 0 ) | (data['DamagePg'] == 0)) & (data['Wins'] >= 1)).value_counts()

print(Zero_Kills)
print(Zero_Damage)
print(Zero_Kills_or_Damage)
print(Zero_Kills_or_Damage_Win)

The number of players with zero kills is 1062. Now, the number of players with zero damage is lower with 619. Now for the question whether a player can win without damaging or killing another player in this dataset, and the answer is NO!


Does traveling more distance have a correlation to higher win rate? Let's analyze this by comparing the lower 50% and upper 50% of the population

In [ ]:
## Do people who walk more than 50% of the population have higher win ratios?
## Calculate upper half?
x = data[(data['AvgTotalDistance'] > data['AvgTotalDistance'].quantile(0.50))]
wins_upper = x['WinRatio'].mean()
# Calculate lower half
x = data[(data['AvgTotalDistance'] < data['AvgTotalDistance'].quantile(0.50))]
wins_lower = x['WinRatio'].mean()
winratio_mean = data['WinRatio'].mean()


sns.set(style="white", rc={"lines.linewidth": 3})
fig, ax1 = plt.subplots(figsize=(4,4))
ax2 = ax1.twinx()
sns.barplot(y = [wins_lower, wins_upper, winratio_mean], x = ['<50%', '>50%', 'Win Ratio Mean'], ax = ax1)
sns.lineplot(x=['<50%',' > 50%','Mean'], 
             #y=[wins_lower, wins_upper, winratio_mean],
             y=[wins_upper , wins_lower, winratio_mean],
             color='r',
             marker="o",
             ax=ax1)
plt.show()
sns.set()

print(wins_lower)
print(wins_upper)
print(winratio_mean)

The lower end of the spectrum has a 2.73 win ratio, the other end of the spectrum has a 7.31 win ratio, and the mean is 5.02 winratio. With this in mind, having a higher average traveling distance can lead to a greater win ratio!

Does surviving time have a correlation to higher win rate? Let's analyze this by comparing the lower 50% and upper 50% of the population

In [ ]:
## Do people who survive longer than 50% of the population have higher win ratios?
## Calculate upper half?
x = data[(data['AvgSurvivalTime'] > data['AvgSurvivalTime'].quantile(0.50))]
wins_upper = x['WinRatio'].mean()
# Calculate lower half
x = data[(data['AvgSurvivalTime'] < data['AvgSurvivalTime'].quantile(0.50))]
wins_lower = x['WinRatio'].mean()
winratio_mean = data['WinRatio'].mean()



sns.set(style="white", rc={"lines.linewidth": 3})
fig, ax1 = plt.subplots(figsize=(4,4))
ax2 = ax1.twinx()
sns.barplot(y = [wins_lower, wins_upper, winratio_mean], x = ['<50%', '>50%', 'Win Ratio Mean'], ax = ax1)
sns.lineplot(x=['<50%',' > 50%','Mean'], 
             #y=[wins_lower, wins_upper, winratio_mean],
             y=[wins_upper , wins_lower, winratio_mean],
             color='purple',
             marker="o",
             ax=ax1)
plt.show()
sns.set()

print(wins_lower)
print(wins_upper)
print(winratio_mean)




Does surviving time have a correlation to higher KDRs? Let's analyze this by comparing the lower 50% and upper 50% of the population

In [ ]:
## Do people who survive longer than 50% of the population have higher KDRs
## Calculate upper half?
x = data[(data['AvgSurvivalTime'] > data['AvgSurvivalTime'].quantile(0.50))]
wins_upper = x['KillDeathRatio'].mean()
# Calculate lower half
x = data[(data['AvgSurvivalTime'] < data['AvgSurvivalTime'].quantile(0.50))]
wins_lower = x['KillDeathRatio'].mean()
winratio_mean =data['KillDeathRatio'].mean()

sns.set(style="white", rc={"lines.linewidth": 3})
fig, ax1 = plt.subplots(figsize=(4,4))
ax2 = ax1.twinx()
sns.barplot(y = [wins_lower, wins_upper, winratio_mean],  x = ['<50%', '>50%', 'Win Ratio Mean'], ax = ax1)
sns.lineplot(x=['<50%',' > 50%','Mean'], 
             #y=[wins_lower, wins_upper, winratio_mean],
             y=[wins_upper , wins_lower, winratio_mean],
             color='purple',
             marker="o",
             ax=ax1)
plt.show()
sns.set()


print(wins_upper)
print(wins_lower)
print(data['KillDeathRatio'].describe())

What factors correlate to a higher average survival time?

In [ ]:
g = sns.pairplot(data,
                 x_vars=["KillsPg", "AvgTotalDistance", 'KillDeathRatio',  'HeadshotKillRatio', 'WinRatio', 'Top10Ratio', 'RoundsPlayed'],
                 y_vars=["AvgSurvivalTime"])

----------------------------------------------------------------------------------------------------------------------

Clustering 3D (Selected Few Features)

In [ ]:
# Select four features
train_data = train.loc[:,['WinRatio', 'KillDeathRatio', "HeadshotKillRatio", "Top10Ratio"]]
test_data = test.loc[:, ['WinRatio', 'KillDeathRatio', "HeadshotKillRatio", "Top10Ratio"]]



In [ ]:
train_data.describe()

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(train_data)
X_test_std = scaler.transform(test_data)


In [ ]:
# K means clustering of Training Data
number_cluster = 3
kmeans = KMeans(n_clusters=number_cluster).fit(X_train_std)
labels = kmeans.labels_

In [ ]:
# Example 3D 
import plotly.express as px
election = px.data.election()
fig = px.scatter_3d(election, x="Joly", y="Coderre", z="Bergeron", color="winner", size="total", hover_name="district",
                  symbol="result", color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"})
fig.show()

In [ ]:
def scatter3d_cluster(df, x, y, z, code ):
    fig = px.scatter_3d(df, x=x, y=y, z=z, color = code)
    return fig.show()

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
df_X_train_std = pd.DataFrame(X_train_std)
df_X_train_std['cluster'] = pd.Series(labels, index=df_X_train_std.index)
df_X_train_std.columns = ['Win Rate', 'KDR', "Headshot Kill Ratio", "Top 10 Ratio", 'cluster']

# Plots of Win Rate, KDR, Headshott KIll Ratio
scatter3d_cluster(df = df_X_train_std , x = 'Win Rate'
                  , y = 'KDR', z = 'Headshot Kill Ratio', code = 'cluster' )

scatter3d_cluster(df = df_X_train_std , x = 'Win Rate'
                  , y = 'KDR', z = 'Top 10 Ratio', code = 'cluster' )


In [ ]:
# # K means clustering of Training Data
predict_labels = kmeans.predict(X_test_std)

In [ ]:
## 3D Plot of Testing Data
# Create and modify dataframe for the cluster column
df_X_test_std = pd.DataFrame(X_test_std)
df_X_test_std['cluster'] = pd.Series(predict_labels, index=df_X_test_std.index)
df_X_test_std.columns = ['Win Rate', 'KDR', "Headshot Kill Ratio", "Top 10 Ratio", 'cluster']

# Plots of Win Rate, KDR, Headshott KIll Ratio
scatter3d_cluster(df = df_X_test_std , x = 'Win Rate'
                  , y = 'KDR', z = 'Headshot Kill Ratio', code = 'cluster' )

scatter3d_cluster(df = df_X_test_std , x = 'Win Rate'
                  , y = 'KDR', z = 'Top 10 Ratio', code = 'cluster' )

PCA 3D Dimensions SELECTED FEW FEATURES

Training

In [ ]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(X_train_std)#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Pulsar Dataset Explained Variance')
plt.show()
# Explains why 2D PCA was better in clustering than 3D

In [ ]:
## Principle Components Analysis
# Set PCA parameters
pca = PCA(n_components=3)
# Fit and transform
principalComponents = pca.fit_transform(X_train_std)
features = range(pca.n_components_)
# Plot the expected variance
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
# Create a dataframe for PCA_components
PCA_components_train = pd.DataFrame(principalComponents)
PCA_components_train.columns = ['PCA 1', 'PCA 2', "PCA 3"]

In [ ]:
PCA_components_train.columns = ['PCA 1', 'PCA 2', "PCA 3"]
scatter3d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', z = 'PCA 3', code = None)

plt.scatter(PCA_components_train['PCA 1'], PCA_components_train['PCA 2'], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(PCA_components_train.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# K means train
number_cluster = 3
kmeans = KMeans(n_clusters=number_cluster).fit(principalComponents)
labels = kmeans.labels_

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
PCA_components_train['cluster'] = pd.Series(labels, index=PCA_components_train.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter3d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', z = 'PCA 3', code = 'cluster')

Testing

In [ ]:
# Transform the test set using the training set PCA fit
principalComponents_test = pca.transform(X_test_std)


# Create a dataframe for PCA_components
PCA_components_test = pd.DataFrame(principalComponents_test)
PCA_components_test.columns = ['PCA 1', 'PCA 2', "PCA 3"]


In [ ]:
# # K means clustering of Testing Data
predict_labels = kmeans.predict(principalComponents_test)

In [ ]:
PCA_components_test['cluster'] = pd.Series(predict_labels, index=PCA_components_test.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter3d_cluster(df = PCA_components_test , x = 'PCA 1'
                  , y = 'PCA 2', z = 'PCA 3', code = 'cluster')


Clustering 2D PCA 2D 

Clustering, Select two variables: Headshot and Kill-Death Ratio

In [ ]:
train_data = train.loc[:,['WinRatio', 'KillDeathRatio', "HeadshotKillRatio", "Top10Ratio"]]
test_data = test.loc[:, ['WinRatio', 'KillDeathRatio', "HeadshotKillRatio", "Top10Ratio"]]


In [ ]:
print(train_data.describe())
print(len(test_data))

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(train_data)
X_test_std = scaler.transform(test_data)


In [ ]:
# K means clustering of Training Data
number_cluster = 2
kmeans = KMeans(n_clusters=number_cluster).fit(X_train_std)
labels = kmeans.labels_

In [ ]:
def scatter2d_cluster(df, x, y, code ):
    fig = px.scatter(df, x=x, y=y, color = code)
    return fig.show()

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
df_X_train_std = pd.DataFrame(X_train_std)
df_X_train_std['cluster'] = pd.Series(labels, index=df_X_train_std.index)
df_X_train_std.columns = ['Win Rate', 'KDR', "Headshot Kill Ratio", "Top 10 Ratio", 'cluster']

# Plots of Win Rate, KDR, Headshott KIll Ratio
scatter2d_cluster(df = df_X_train_std , x = 'KDR'
                  , y = 'Win Rate',  code = 'cluster' )

scatter2d_cluster(df = df_X_train_std , x = 'KDR'
                  , y = 'Headshot Kill Ratio', code = 'cluster' )

scatter2d_cluster(df = df_X_train_std , x = 'Headshot Kill Ratio'
                  , y = 'Win Rate',  code = 'cluster' )

scatter2d_cluster(df = df_X_train_std , x = 'KDR'
                  , y = 'Top 10 Ratio', code = 'cluster' )

scatter2d_cluster(df = df_X_train_std , x = 'Headshot Kill Ratio'
                  , y = 'Top 10 Ratio', code = 'cluster' )


In [ ]:
# # K means clustering of Training Data
predict_labels = kmeans.predict(X_test_std)

In [ ]:
## 3D Plot of Testing Data
# Create and modify dataframe for the cluster column
df_X_test_std = pd.DataFrame(X_test_std)
df_X_test_std['cluster'] = pd.Series(predict_labels, index=df_X_test_std.index)
df_X_test_std.columns = ['Win Rate', 'KDR', "Headshot Kill Ratio", "Top 10 Ratio", 'cluster']

# Plots of Win Rate, KDR, Headshott KIll Ratio
scatter2d_cluster(df = df_X_test_std , x = 'KDR'
                  , y = 'Win Rate',  code = 'cluster' )

scatter2d_cluster(df = df_X_test_std , x = 'KDR'
                  , y = 'Headshot Kill Ratio', code = 'cluster' )

scatter2d_cluster(df = df_X_test_std , x = 'Headshot Kill Ratio'
                  , y = 'Win Rate',  code = 'cluster' )

scatter2d_cluster(df = df_X_test_std , x = 'KDR'
                  , y = 'Top 10 Ratio', code = 'cluster' )

scatter2d_cluster(df = df_X_test_std , x = 'Headshot Kill Ratio'
                  , y = 'Top 10 Ratio', code = 'cluster' )

PCA 2D Dimensions SELECT FEW Features

Training

In [ ]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(X_train_std)#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Pulsar Dataset Explained Variance')
plt.show()
# Explains why 2D PCA was better in clustering than 3D

In [ ]:
## Principle Components Analysis
# Set PCA parameters
pca = PCA(n_components=2)
# Fit and transform
principalComponents = pca.fit_transform(X_train_std)
features = range(pca.n_components_)
# Plot the expected variance
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
# Create a dataframe for PCA_components
PCA_components_train = pd.DataFrame(principalComponents)
PCA_components_train.columns = ['PCA 1', 'PCA 2']

In [ ]:
PCA_components_train.columns = ['PCA 1', 'PCA 2']
scatter2d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', code = None)

plt.scatter(PCA_components_train['PCA 1'], PCA_components_train['PCA 2'], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(PCA_components_train.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# K means train #2
number_cluster = 2
kmeans = KMeans(n_clusters=number_cluster).fit(principalComponents)
labels = kmeans.labels_

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
PCA_components_train['cluster'] = pd.Series(labels, index=PCA_components_train.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter2d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', code = 'cluster')

Testing

In [ ]:
# Transform the test set using the training set PCA fit
principalComponents_test = pca.transform(X_test_std)


# Create a dataframe for PCA_components
PCA_components_test = pd.DataFrame(principalComponents_test)
PCA_components_test.columns = ['PCA 1', 'PCA 2']


In [ ]:
# # K means clustering of Testing Data
predict_labels = kmeans.predict(principalComponents_test)

In [ ]:
PCA_components_test['cluster'] = pd.Series(predict_labels, index=PCA_components_test.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter2d_cluster(df = PCA_components_test , x = 'PCA 1'
                  , y = 'PCA 2',  code = 'cluster')


PCA ALL FEATURES 2D

In [ ]:
train_data = train
test_data = test




In [ ]:
# Drop Revives because this is solo
train_data.drop(columns = ['Revives'], inplace=True)
test_data.drop(columns = ['Revives'], inplace=True)

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(train_data)
X_test_std = scaler.transform(test_data)


In [ ]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(X_train_std)#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Pulsar Dataset Explained Variance')
plt.show()

In [ ]:
## Principle Components Analysis
# Set PCA parameters
pca = PCA(n_components=30)
# Fit and transform
principalComponents = pca.fit_transform(X_train_std)
features = range(pca.n_components_)
# Plot the expected variance
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
# Create a dataframe for PCA_components
PCA_components_train = pd.DataFrame(principalComponents)
PCA_components_train.columns = ['PCA' + ' ' + str(i+1) for i in features]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
print((pca.explained_variance_ratio_).sum())


In [ ]:
scatter2d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', code = None)

plt.scatter(PCA_components_train['PCA 1'], PCA_components_train['PCA 2'], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(PCA_components_train.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# K means train #2
number_cluster = 2
kmeans = KMeans(n_clusters=number_cluster).fit(principalComponents)
labels = kmeans.labels_

In [ ]:
def scatter2d_cluster(df, x, y, code ):
    fig = px.scatter(df, x=x, y=y, color = code)
    return fig.show()

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
PCA_components_train['cluster'] = pd.Series(labels, index=PCA_components_train.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter2d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', code = 'cluster')

Testing

In [ ]:
# Transform the test set using the training set PCA fit
principalComponents_test = pca.transform(X_test_std)


# Create a dataframe for PCA_components
PCA_components_test = pd.DataFrame(principalComponents_test)
PCA_components_test.columns = ['PCA' + ' ' + str(i+1) for i in features]


In [ ]:
# # K means clustering of Testing Data
predict_labels = kmeans.predict(principalComponents_test)

In [ ]:
PCA_components_test['cluster'] = pd.Series(predict_labels, index=PCA_components_test.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter2d_cluster(df = PCA_components_test , x = 'PCA 1'
                  , y = 'PCA 2',  code = 'cluster')


3D

In [ ]:
def scatter3d_cluster(df, x, y, z, code ):
    fig = px.scatter_3d(df, x=x, y=y, z=z, color = code)
    return fig.show()

In [ ]:
## 3D Plot of Training Data
# Create and modify dataframe for the cluster column
PCA_components_train['cluster'] = pd.Series(labels, index=PCA_components_train.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter3d_cluster(df = PCA_components_train , x = 'PCA 1'
                  , y = 'PCA 2', z = 'PCA 3', code = 'cluster')

In [ ]:
PCA_components_test['cluster'] = pd.Series(predict_labels, index=PCA_components_test.index)

# Plots of PCA 1, PCA 2, PCA 3
scatter3d_cluster(df = PCA_components_test , x = 'PCA 1'
                  , y = 'PCA 2', z = 'PCA 3', code = 'cluster')
